# many-to-many

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
string = 'hello world'
labels = 'ello world ' # input을 왼쪽으로 shfit한 것 같은 모양

In [4]:
X_train = list(string)
print(X_train)
y_train = list(labels)
print(y_train)

['h', 'e', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd']
['e', 'l', 'l', 'o', ' ', 'w', 'o', 'r', 'l', 'd', ' ']


In [5]:
idx2char = [' ','d','e','h','l','o','r','w'] # LabelBInarizer는 알파벳 순서로 sort

from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
lb.fit(X_train)
X_train = lb.transform(X_train)
print(X_train)
print(X_train.shape)
print(lb.inverse_transform(X_train))

y_train = lb.transform(y_train)
print(y_train)
print(lb.inverse_transform(y_train))

[[0 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0]]
(11, 8)
['h' 'e' 'l' 'l' 'o' ' ' 'w' 'o' 'r' 'l' 'd']
[[0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]]
['e' 'l' 'l' 'o' ' ' 'w' 'o' 'r' 'l' 'd' ' ']


In [6]:
# RNN의 input: (batch_size, seq_length, input_dim)
# RNN의 output: (batch_size, seq_legth, n_directions * hidden_size), (num_layers * n_directions, batch_size, hidden_size)

batch_size = 1
seq_length = 11 # hello world
input_size = 8
hidden_size = 8 # 출력을 one-hot-encoding의 dim과 맞춘다

X_train = torch.Tensor(X_train)
X_train = X_train.unsqueeze(0)
print(X_train.shape)
y_train = torch.Tensor(y_train)
y_train = y_train.unsqueeze(0)
print(y_train.shape)

torch.Size([1, 11, 8])
torch.Size([1, 11, 8])


In [7]:
rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True)
hidden = torch.zeros(1, batch_size, hidden_size) # 1 = num_layers * n_directions
y, hidden = rnn(X_train, hidden)

print(y.shape, hidden.shape)

torch.Size([1, 11, 8]) torch.Size([1, 1, 8])


In [8]:
# RNN 학습

rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True)

criterion = nn.MSELoss()
optimizer = optim.Adam(rnn.parameters(),lr = 0.1)

for epoch in range(1000):
    optimizer.zero_grad()
    loss = 0
    hidden = torch.zeros(1, batch_size, hidden_size) # 1 = num_layers * n_directions

    y, hidden = rnn(X_train, hidden)
    loss = criterion(y, y_train) # y값 전체(즉 time step에 따른 hidden=output 전체)를 y_train과 비교한다.

    if epoch % 100 == 0:
        print("epoch: %d, loss: %1.3f" % (epoch+1,loss.data))
        _, idx = y.max(-1)
        result_str = [idx2char[c] for c in idx.squeeze()]
        print("Predicted string: ", ''.join(result_str))
    loss.backward()
    optimizer.step()

epoch: 1, loss: 0.187
Predicted string:  wrdddd drdd
epoch: 101, loss: 0.004
Predicted string:  ello world 
epoch: 201, loss: 0.003
Predicted string:  ello world 
epoch: 301, loss: 0.002
Predicted string:  ello world 
epoch: 401, loss: 0.006
Predicted string:  ello world 
epoch: 501, loss: 0.002
Predicted string:  ello world 
epoch: 601, loss: 0.001
Predicted string:  ello world 
epoch: 701, loss: 0.001
Predicted string:  ello world 
epoch: 801, loss: 0.001
Predicted string:  ello world 
epoch: 901, loss: 0.001
Predicted string:  ello world 


In [9]:
# Fully-Connected Layer 학습

fc = nn.Linear(batch_size * input_size * seq_length, batch_size * seq_length * hidden_size)

criterion = nn.MSELoss()
optimizer = optim.Adam(fc.parameters(),lr = 0.1)

for epoch in range(1000):
    optimizer.zero_grad()
    loss = 0

    y = fc(X_train.view(-1, batch_size * input_size * seq_length)) # X_train이 입력될 때 y_train이 나오도록 regression 학습한다.
    y = y.view(batch_size, seq_length, hidden_size)
    loss = criterion(y, y_train)

    if epoch % 100 == 0:
        print("epoch: %d, loss: %1.3f" % (epoch+1,loss.data))
        _, idx = y.max(-1)
        result_str = [idx2char[c] for c in idx.squeeze()]
        print("Predicted string: ", ''.join(result_str))
    loss.backward()
    optimizer.step()

epoch: 1, loss: 0.179
Predicted string:  wowolohlhhw
epoch: 101, loss: 0.000
Predicted string:  ello world 
epoch: 201, loss: 0.000
Predicted string:  ello world 
epoch: 301, loss: 0.000
Predicted string:  ello world 
epoch: 401, loss: 0.000
Predicted string:  ello world 
epoch: 501, loss: 0.000
Predicted string:  ello world 
epoch: 601, loss: 0.000
Predicted string:  ello world 
epoch: 701, loss: 0.000
Predicted string:  ello world 
epoch: 801, loss: 0.000
Predicted string:  ello world 
epoch: 901, loss: 0.000
Predicted string:  ello world 


In [10]:
# CrossEntroryLoss

y_train = list(labels)
y_train = [idx2char.index(x) for x in y_train] 
y_train = torch.LongTensor(y_train)

rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(),lr = 0.1)

for epoch in range(1000):
    optimizer.zero_grad()
    loss = 0
    hidden = torch.zeros(batch_size, 1, hidden_size) # 1 = num_layers * n_directiyyons

    y, hidden = rnn(X_train, hidden)
    y = y.view(-1, input_size) # 결과를 classification의 확률 배열로 보아서 CrossEntropyLoss로 loss 계산
    loss = criterion(y, y_train)

    if epoch % 100 == 0:
        print("epoch: %d, loss: %1.3f" % (epoch+1,loss.data))
        _, idx = y.max(-1)
        result_str = [idx2char[c] for c in idx.squeeze()]
        print("Predicted string: ", ''.join(result_str))
    loss.backward()
    optimizer.step()

epoch: 1, loss: 2.192
Predicted string:  eeewhhhhhhh
epoch: 101, loss: 0.716
Predicted string:  ello world 
epoch: 201, loss: 0.776
Predicted string:  ell  world 
epoch: 301, loss: 0.803
Predicted string:  ell  world 
epoch: 401, loss: 0.800
Predicted string:  ell  world 
epoch: 501, loss: 0.721
Predicted string:  ello world 
epoch: 601, loss: 0.698
Predicted string:  ello world 
epoch: 701, loss: 0.695
Predicted string:  ello world 
epoch: 801, loss: 0.694
Predicted string:  ello world 
epoch: 901, loss: 0.693
Predicted string:  ello world 


## X_train만으로 학습하므로 모델의 성능은 확인할 수 없음, test loss를 확인해야 함